In [40]:
# INITS + IMPORTS
from typing import List
from openai import OpenAI
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from dotenv import load_dotenv

BASE_URL = "http://199.94.61.113:8000/v1/"
API_KEY=api_key="yen.k@northeastern.edu:79wlaJxIMpzvkt61S8Xi"
assert BASE_URL is not None
assert API_KEY is not None

client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

load_dotenv() 

# Set up the client id and client secret (you can find the client secret 
# from the spotify dev page)
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
REDIRECT_URI = 'http://localhost:8888/callback'
SCOPE = 'playlist-modify-public user-modify-playback-state user-top-read'

# Create a spotify object passing the client id, client secret, 
# redirct url (which doesn't matter, just set it as your local host
# as shown below), and scope
spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE,
                                               cache_path=".cache-<your-username>"))

# TODO define the data classes

In [115]:
class SpotifyAgent:
    conversation: List[dict]
    client: OpenAI
    spotify: SpotifyOAuth
    playlist_id: str
    seed_artists: List[str]
    seed_tracks: List[str]
    seed_genres: List[str]
    possible_genres: List[str]
    system_prompt = """
You are a helpful Spotify chatbot. Respond to queries with a single Python block of code that uses
the following function:
def add_to_seed_artists(artist_name):
    ...
def add_to_seed_tracks(track_name):
    ...
def add_to_seed_genres(genre_name):
    ...
Return the result in a variable called result.
Do not redefine the functin stubs just use this existing method.
Don't correct their questions.
"""
    few_shot_prompt = "I like the artist taylor swift."
    few_shot_response = """
```python
result = add_to_seed_artists("taylor swift")
print(result)
```
"""
    few_shot_prompt2 = "I like selena gomez. Get me reccomendations."
    few_shot_response2 = """
```python
result = get_recommendations()
print(result)
```
"""
    
    # current_mood: int # TODO: figure this out

    # clear playlist if exists, else create a new one
    def create_playlist(self):
        user_id = self.spotify.current_user()['id']  
        playlist_name = 'Spotify Chatbot Session'
        playlist_description = 'A playlist created by the Spotify Chatbot.'
        # Get the current user's playlists
        playlists = self.spotify.current_user_playlists()

        # Look for the playlist by name
        existing_playlist = None
        for playlist in playlists['items']:
            if playlist['name'] == playlist_name:
                existing_playlist = playlist
                break

        if existing_playlist:
            # If the playlist exists, take action: delete all tracks from it
            playlist_id = existing_playlist['id']
            print(f"Found existing playlist '{playlist_name}', clearing tracks...")
            
            # Get all tracks
            track_uris = []
            results = self.spotify.playlist_items(playlist_id)
            track_uris.extend([item['track']['uri'] for item in results['items']])

            # Remove all tracks
            if track_uris:
                self.spotify.playlist_remove_all_occurrences_of_items(playlist_id, track_uris)
                print(f"All tracks removed from playlist '{playlist_name}'.")

            return playlist_id
        else:
            # If the playlist does not exist, create a new one
            print(f"Playlist '{playlist_name}' does not exist. Creating a new one...")
            res = self.spotify.user_playlist_create(user=user_id, 
                                    name=playlist_name, 
                                    public=True, 
                                    description=playlist_description)
            print(f"Playlist '{playlist_name}' created successfully.")
            return res['id']

    # Add one song or multiple songs to current playlist
    def add_tracks_to_playlist(self, tracks):
        track_uris = []
        for track in tracks:
            track_uris.append(track['uri'])
        spotify.playlist_add_items(self.playlist_id, track_uris)

    # Get recommendations based on seeds
    def get_recommendations(self):
        limit = 10
        # if all seeds are none, return top songs
        if len(self.seed_artists) == 0 and len(self.seed_tracks) == 0 and len(self.seed_genres) == 0:
            print("No seeds provided. Returning top tracks.")
            return spotify.current_user_top_tracks(limit=limit, offset=0, time_range='short_term')['items']
        
        recommendations = spotify.recommendations(
            seed_artists=self.seed_artists,
            seed_tracks=self.seed_tracks,
            seed_genres=self.seed_genres,
            limit=limit
        )

        return recommendations['tracks']
    
    def add_to_seed_artists(self, artist_name):
        # Perform a search query for the artist
        results = spotify.search(q='artist:' + artist_name, type='artist')
        
        items = results['artists']['items']
        if items:
            artist = items[0]
            self.seed_artists.append(artist['uri']) # add first artist
        
         # TODO Figure out error handling

    def add_to_seed_tracks(self, track_name):
        results = spotify.search(q='track:' + track_name, type='track')
        items = results['tracks']['items']
        if items:
            track = items[0]
            self.seed_tracks.append(track['uri'])
    
    def add_to_seed_genres(self, genre_name):
        print('possible genres', self.possible_genres)
        if genre_name.lower() in self.possible_genres:
            self.seed_genres.append(genre_name.lower())

    def extract_code(self, resp_text):
        code_start = resp_text.find("```")
        code_end = resp_text.rfind("```")
        if code_start == -1 or code_end == -1:
            return "pass"
        
        return resp_text[code_start + 3 + 7:code_end]
    
    def run_code(self, code_text):
        globals = { 
            "add_tracks_to_playlist": self.add_tracks_to_playlist, 
            "get_recommendations": self.get_recommendations,
            "add_to_seed_artists": self.add_to_seed_artists,
            "add_to_seed_tracks": self.add_to_seed_tracks,
            "add_to_seed_genres": self.add_to_seed_genres
        }
        exec(code_text, globals)
        return globals["result"]
    
    # TODO add the types??
    def say(self, user_message: str):
        # if len(self.conversation) > 7:
        #     self.conversation.pop(1)
        #     self.conversation.pop(2)
        # Add the user message to the conversation.
        self.conversation.append({"role": "user", "content": user_message})
        
        # Get the response from the model.
        resp = self.client.chat.completions.create(
            messages = self.conversation,
            model = "meta-llama/Meta-Llama-3.1-8B-Instruct",
            temperature=0)
        
        resp_text = resp.choices[0].message.content

        self.conversation.append({"role": "system", "content": resp_text })
        code_text = self.extract_code(resp_text)
        # TODO if prompt was relating to add seed artists, delete from conversation
        # store conversation to display
        res = self.run_code(code_text)
        print('CODE TEXT', code_text)
        if "get_recommendations" in code_text:
            print('RECOMMENDATIONS')
            for track in res:
                print(track['name'], 'by', track['artists'][0]['name'])
        if "add_to_seed_artists" in code_text:
            print('ARTIST SEEDS', self.seed_artists)
            # TODO: we need to do something with the result
        if "add_to_seed_tracks" in code_text:
            print('TRACK SEEDS', self.seed_tracks)
        if "add_to_seed_genres" in code_text:
            print('GENRE SEEDS', self.seed_genres)
        print(res)
    
    def __init__(self, client: OpenAI, spotify: SpotifyOAuth):
        self.client = client
        self.spotify = spotify
        self.playlist_id = self.create_playlist() # Create a playlist for the session
        self.possible_genres = spotify.recommendation_genre_seeds()['genres']
        self.conversation = [{ "role": "system", "content": self.system_prompt },
                             { "role": "user", "content": self.few_shot_prompt },
                             { "role": "system", "content": self.few_shot_response }, 
                             { "role": "user", "content": self.few_shot_prompt2 },
                             { "role": "system", "content": self.few_shot_response2 }]
        self.seed_artists = []
        self.seed_tracks = []
        self.seed_genres = []
        # self.current_mood = 0


In [114]:
agent = SpotifyAgent(client, spotify)

res = agent.say("I like the hollie col.")
res = agent.say("I like the artist taylor swift.")
res = agent.say("I like punk music")
res = agent.say("get me some recomendations")

# TODO: figure the best way to add song to playlist 
# TODO; not accounting for spelling errors?
# TODO: figure out how to handle mood
# TODO: handling artists that aren't so well known
# TODO: handle removing from the playlist
# TODO: add more complex functionality
# Add all the variable stuff similar to the travel agent, also add benchmark tests.

Found existing playlist 'Spotify Chatbot Session', clearing tracks...
None
CODE TEXT result = add_to_seed_artists("hollie col")
print(result)

ARTIST SEEDS ['spotify:artist:4JyT0CxxEic1JhENHbXfR1']
None
None
CODE TEXT result = add_to_seed_artists("taylor swift")
print(result)

ARTIST SEEDS ['spotify:artist:4JyT0CxxEic1JhENHbXfR1', 'spotify:artist:06HL4z0CvFAxyc27GXpf02']
None
possible genres ['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holid